## Deliver Time
### v1 2019-05-29

In [1]:
import json
import numpy as np
from scipy import stats
from datetime import datetime

import pandas as pd
pd.options.display.max_columns = None

import analitico
import analitico.plugin
import s24.plugin
from analitico.pandas import *
from analitico.schema import apply_schema

# pass api token to create sdk
sdk = analitico.authorize("tok_s24_579E5hOWw7k8")
print(datetime.utcnow())

2019-05-29 14:55:54.833920


## Training

In [2]:
# load table that is already a result of joining and processing several tables
df1 = sdk.get_dataframe("ds_s24_order_time_pick_pay_deliver")
df1.sample(n=10)

,order_amount,order_volume,order_deliver_at_start.dayofweek,order_deliver_at_start.year,order_deliver_at_start.month,order_deliver_at_start.day,order_deliver_at_start.hour,order_deliver_at_start.minute,order_deliver_at_end,order_fulfillment_type,items_total,items_with_variable_weight,store_name,store_province,store_lat,store_lng,store_area,store_ref_id,customer_province,customer_lat,customer_lng,customer_area,customer_ztl,customer_ref_id,customer_has_subscription,pick_time.min,pay_time.min,deliver_time.min
78958,64.43,39.17,2,2017,12,20,9,0,2017-12-20 10:00:00,1,15,3,conad,TV,45.649091,12.239769,TV1,2470,TV,45.670688,12.232729,TV1,0,693523,0,23.033333,6.000000,18.183333
148851,64.05,45.68,2,2018,6,27,17,0,2018-06-27 18:00:00,1,15,1,coop,BG,45.682268,9.669653,BG1,2093,BG,45.705580,9.685756,BG1,0,958769,0,13.150000,7.900000,25.500000
77790,59.37,68.53,5,2017,12,16,10,0,2017-12-16 11:00:00,1,24,5,mercatò,TO,45.063979,7.638144,TO2,1295,TO,45.061174,7.673081,TO2,0,782595,0,12.333333,21.200000,16.566667
220112,31.71,32.88,4,2018,11,9,11,0,2018-11-09 12:00:00,1,12,0,bennet,FC,44.191067,12.117496,FC1,3614,FC,44.189600,12.154580,FC1,0,1276415,0,17.983333,29.733333,15.400000
19726,40.63,0.00,4,2016,12,9,18,0,2016-12-09 19:00:00,1,13,0,ipercoop,MI,45.448101,9.211625,NaN,4494,MI,45.454286,9.195609,NaN,0,415594,0,31.600000,2.350000,NaN
154016,59.14,56.30,6,2018,7,8,14,0,2018-07-08 15:00:00,1,24,6,ipercoop,MI,45.519752,9.149729,MI2,4497,MI,45.510600,9.199990,MI2,0,645872,0,28.250000,7.216667,107.000000
172673,20.45,45.41,3,2018,8,23,14,0,2018-08-23 15:00:00,1,7,0,famila,VR,45.437310,10.760050,VR4,2749,VR,45.429296,10.671195,VR4,0,909661,0,7.216667,16.633333,21.466667
191661,17.01,11.33,3,2018,9,27,16,0,2018-09-27 17:00:00,1,2,1,bennet,TO,45.095055,7.671651,TO1,1198,TO,45.106204,7.626981,TO1,0,669985,1,49.750000,4.016667,38.433333
196562,58.76,46.31,4,2018,10,5,17,0,2018-10-05 18:00:00,1,17,5,esselunga,BS,45.455012,10.522403,BS2,2188,BS,45.488138,10.519645,BS2,0,944897,0,11.416667,19.500000,16.300000
237075,58.84,101.59,5,2018,12,1,18,0,2018-12-01 19:00:00,1,33,6,eurospin,RM,41.831962,12.582428,RM9,5031,RM,41.860398,12.618254,RM9,0,914089,0,15.800000,8.483333,28.400000


In [3]:
df1.describe()

,order_amount,order_volume,order_deliver_at_start.dayofweek,order_deliver_at_start.year,order_deliver_at_start.month,order_deliver_at_start.day,order_deliver_at_start.hour,order_deliver_at_start.minute,items_total,items_with_variable_weight,store_lat,store_lng,customer_lat,customer_lng,pick_time.min,pay_time.min,deliver_time.min
count,280025.000000,280025.000000,280025.000000,280025.000000,280025.000000,280025.000000,280025.000000,280025.000000,280025.000000,280025.000000,280025.000000,280025.000000,280025.000000,280025.000000,280025.000000,280025.000000,195425.000000
mean,57.065407,75.075223,2.710740,2017.710915,6.988594,15.698448,14.351422,0.138959,22.008724,3.039925,44.142540,10.625622,44.142049,10.629580,25.914597,13.903314,27.464454
std,40.018748,65.127621,1.887745,0.718980,3.684170,8.593538,3.337425,2.046893,14.786909,3.702682,1.710798,1.961118,1.709494,1.960951,32.759049,10.613576,17.179451
min,0.010000,0.000000,0.000000,2016.000000,1.000000,1.000000,7.000000,0.000000,1.000000,0.000000,37.511822,7.401420,37.434983,7.352680,1.000000,1.000000,1.000000
25%,30.730000,30.210000,1.000000,2017.000000,4.000000,8.000000,11.000000,0.000000,11.000000,0.000000,41.927061,9.121973,41.929204,9.131372,11.433333,7.883333,16.316667
50%,47.350000,62.700000,3.000000,2018.000000,7.000000,16.000000,14.000000,0.000000,19.000000,2.000000,45.063979,10.994936,45.065830,11.002461,20.133333,11.200000,22.816667
75%,72.990000,103.410000,4.000000,2018.000000,10.000000,23.000000,17.000000,0.000000,29.000000,5.000000,45.456849,12.452055,45.448825,12.463640,33.033333,16.066667,32.933333
max,934.850000,1214.980000,6.000000,2019.000000,12.000000,31.000000,22.000000,55.000000,251.000000,42.000000,46.607851,15.169497,46.099460,26.977770,4368.883333,89.950000,119.983333


In [4]:
# keep only the features we need, apply data types
df2 = apply_schema(df1, {
    "columns":[
        { "name":"order_amount", "type":"float" },
        { "name":"order_volume", "type":"float" },
        { "name":"order_deliver_at_start.dayofweek", "type":"integer" },
        { "name":"order_deliver_at_start.year", "type":"integer" },
        { "name":"order_deliver_at_start.month", "type":"integer" },
        { "name":"order_deliver_at_start.day", "type":"integer" },
        { "name":"order_deliver_at_start.hour", "type":"integer" },
        { "name":"order_deliver_at_start.minute", "type":"integer" },
        { "name":"order_fulfillment_type", "type":"category" },
        { "name":"items_total", "type":"integer" },
        { "name":"items_with_variable_weight", "type":"integer" },
        { "name":"store_name", "type":"category" },
        { "name":"store_province", "type":"category" },
        { "name":"store_lat", "type":"float" },
        { "name":"store_lng", "type":"float" },
        { "name":"store_area", "type":"category" },
        { "name":"store_ref_id", "type":"category" },
        { "name":"customer_province", "type":"category" },
        { "name":"customer_lat", "type":"float" },
        { "name":"customer_lng", "type":"float" },
        { "name":"customer_area", "type":"category" },
        { "name":"customer_ztl", "type":"category" },
        { "name":"customer_ref_id", "type":"category" },
        { "name":"customer_has_subscription", "type":"category" },
        { "name":"pay_time.min", "type":"float" }
    ]
})
df2.sample(n=10)

,order_amount,order_volume,order_deliver_at_start.dayofweek,order_deliver_at_start.year,order_deliver_at_start.month,order_deliver_at_start.day,order_deliver_at_start.hour,order_deliver_at_start.minute,order_fulfillment_type,items_total,items_with_variable_weight,store_name,store_province,store_lat,store_lng,store_area,store_ref_id,customer_province,customer_lat,customer_lng,customer_area,customer_ztl,customer_ref_id,customer_has_subscription,pay_time.min
109899,41.45,37.09,2,2018,3,28,20,0,1,18,2,esselunga,TO,45.021490,7.652017,TO2,1287,TO,45.017833,7.679477,TO2,0,807421,0,16.533333
42217,110.39,158.04,0,2017,5,22,16,0,1,27,6,conad,RM,41.876030,12.443046,RM4,2899,RM,41.878932,12.458121,NaN,0,358994,0,22.950000
96419,50.61,64.58,0,2018,2,19,17,0,1,27,1,cadoro,TV,45.685200,12.255370,TV1,284,TV,45.665124,12.246969,TV1,1,258686,0,6.450000
119263,46.39,65.43,6,2018,4,22,15,0,1,32,0,famila superstore,VA,45.828140,8.835571,VA1,1584,VA,45.829884,8.823242,VA1,0,818351,0,5.666667
132963,61.67,92.70,5,2018,5,26,12,0,1,23,8,esselunga,MI,45.537280,9.230243,MI10,4464,MI,45.558364,9.201371,MI10,0,481928,0,20.950000
257109,81.78,125.38,5,2018,12,29,12,0,1,61,11,famila,MN,45.143088,10.763804,MN1,3394,MN,45.134649,10.718629,MN1,0,1210231,0,19.116667
212516,27.79,54.09,0,2018,10,29,15,0,1,8,0,ipercoop,MI,45.448233,9.211739,MI4,4494,MI,45.450890,9.224737,MI4,0,864747,0,4.466667
102960,73.60,204.74,4,2018,3,9,15,0,1,34,8,eurospin,RM,41.895862,12.569003,RM2,5030,RM,41.874572,12.546457,RM2,0,463922,0,9.433333
100475,31.56,44.00,5,2018,3,3,10,0,1,22,3,mercatò,TO,45.063979,7.638144,TO2,1295,TO,45.056899,7.662934,TO2,0,456730,0,13.833333
62553,109.62,114.07,5,2017,10,21,16,0,1,42,16,mercatò,TO,45.063979,7.638144,TO2,1295,TO,45.068270,7.688701,TO2,1,655604,0,27.300000


In [5]:
# remove rows with missing values
df2 = df2[df2["store_area"].notnull()]
df2 = df2[df2["customer_area"].notnull()]

In [6]:
# train catboost sdg model, save to disk, save model metadata
training = sdk.run_plugin(df2, action="train", settings = {
    "name": "analitico.plugin.CatBoostClassifierPlugin",
        "parameters": {
          "iterations": 50,
          "learning_rate": 1
        },
        "data": {
          "label": "pay_time.min"
        }
    })

0:	learn: 10.8773771	test: 10.7228766	best: 10.7228766 (0)	total: 217ms	remaining: 10.6s
1:	learn: 10.7871637	test: 10.6449556	best: 10.6449556 (1)	total: 389ms	remaining: 9.33s
2:	learn: 10.7574109	test: 10.6162432	best: 10.6162432 (2)	total: 552ms	remaining: 8.65s
3:	learn: 10.7330400	test: 10.6064804	best: 10.6064804 (3)	total: 717ms	remaining: 8.25s
4:	learn: 10.7156697	test: 10.6000458	best: 10.6000458 (4)	total: 895ms	remaining: 8.06s
5:	learn: 10.7084743	test: 10.6011701	best: 10.6000458 (4)	total: 1.04s	remaining: 7.66s
6:	learn: 10.6963916	test: 10.5982519	best: 10.5982519 (6)	total: 1.25s	remaining: 7.68s
7:	learn: 10.6907499	test: 10.5951087	best: 10.5951087 (7)	total: 1.41s	remaining: 7.38s
8:	learn: 10.6812220	test: 10.5916414	best: 10.5916414 (8)	total: 1.59s	remaining: 7.23s
9:	learn: 10.6699194	test: 10.5867047	best: 10.5867047 (9)	total: 1.77s	remaining: 7.1s
10:	learn: 10.6620434	test: 10.5871680	best: 10.5867047 (9)	total: 1.92s	remaining: 6.8s
11:	learn: 10.6535099	

## Prediction

In [7]:
import analitico
import analitico.pandas
import pandas as pd

sdk = analitico.authorize("tok_s24_579E5hOWw7k8")

# define method used for serverless deployment of predictions
def handle(event, **kwargs):
    # convert records to pandas dataframe if needed
    if not isinstance(event, pd.DataFrame):
        event = pd.DataFrame.from_dict(event)
    
    if "order_deliver_at_start" in event.columns:
        event = analitico.pandas.augment_dates(event, column="order_deliver_at_start")
    
    # use trained model for inference
    return sdk.run_plugin(event, action="predict", settings = {
        "name": "analitico.plugin.CatBoostClassifierPlugin"
        })

## Testing

In [8]:
# extract some sample records used to test predictions
samples = df2.sample(n=3)
samples = samples.to_dict(orient="records")
print(json.dumps(samples, indent=2))

[
  {
    "order_amount": 58.34,
    "order_volume": 114.57,
    "order_deliver_at_start.dayofweek": 2,
    "order_deliver_at_start.year": 2018,
    "order_deliver_at_start.month": 12,
    "order_deliver_at_start.day": 12,
    "order_deliver_at_start.hour": 15,
    "order_deliver_at_start.minute": 0,
    "order_fulfillment_type": "1",
    "items_total": 16,
    "items_with_variable_weight": 2,
    "store_name": "famila superstore",
    "store_province": "VR",
    "store_lat": 45.46218,
    "store_lng": 10.968289,
    "store_area": "VR3",
    "store_ref_id": "2769",
    "customer_province": "VR",
    "customer_lat": 45.4469728,
    "customer_lng": 10.996176,
    "customer_area": "VR3",
    "customer_ztl": "20",
    "customer_ref_id": "14324",
    "customer_has_subscription": "0",
    "pay_time.min": 12.5
  },
  {
    "order_amount": 34.35,
    "order_volume": 25.01,
    "order_deliver_at_start.dayofweek": 1,
    "order_deliver_at_start.year": 2018,
    "order_deliver_at_start.month": 1,

In [9]:
# run prediction test, print results
results = handle(samples)
print("Prediction results:\n")
print(json.dumps(results, indent=2))

/Users/gionata/github/analitico/venv/lib/python3.7/site-packages/pandas/core/generic.py:5890: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


Prediction results:

{
  "type": "analitico/prediction",
  "performance": {
    "cpu_count": 12,
    "loading_ms": 1,
    "total_ms": 399
  },
  "records": [
    {
      "order_amount": 58.34,
      "order_volume": 114.57,
      "order_deliver_at_start.dayofweek": 2,
      "order_deliver_at_start.year": 2018,
      "order_deliver_at_start.month": 12,
      "order_deliver_at_start.day": 12,
      "order_deliver_at_start.hour": 15,
      "order_deliver_at_start.minute": 0,
      "order_fulfillment_type": "1",
      "items_total": 16,
      "items_with_variable_weight": 2,
      "store_name": "famila superstore",
      "store_province": "VR",
      "store_lat": 45.46218,
      "store_lng": 10.968289,
      "store_area": "VR3",
      "store_ref_id": "2769",
      "customer_province": "VR",
      "customer_lat": 45.446973,
      "customer_lng": 10.996176,
      "customer_area": "VR3",
      "customer_ztl": "20",
      "customer_ref_id": "14324",
      "customer_has_subscription": "0",
     